<a href="https://colab.research.google.com/github/KhaledGhaleb/CorelDataSet/blob/main/Ass_3_Khaled_Fayed_NoteBook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment 3**

Get Corel Reduced Data from Github

In [ ]:
import tensorflow as tf
import os
_URL = 'https://raw.githubusercontent.com/KhaledGhaleb/CorelDataSet/main/CORELREduced.zip'
path_to_zip = tf.keras.utils.get_file('COREL-REduced.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'COREL-REduced')

# 1- Pretrained VGG16 

Prepare Train and Validation data:

set ratio of 0.5 validation = 20 * 0.5 = 10 validation

Set Image size to 224*224 * 3 to support VGG16


In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
BATCH_SIZE = 128
IMG_SIZE = (224, 224)
COLOR_MODE = 'rgb' #"grayscale"  #  "rgb"
Color_Num = 3
Valid_ratio = 0.5

train_dataset_VGG16 = image_dataset_from_directory(PATH,
                                             seed = 1234,                                           
                                             color_mode = COLOR_MODE, 
                                             validation_split = Valid_ratio,
                                             subset = "training",
                                             image_size=IMG_SIZE,
                                             batch_size=BATCH_SIZE,
                                             label_mode ='categorical',
                                             )
Validation_dataset_VGG16 = image_dataset_from_directory(PATH,
                                             seed = 1234,                                           
                                             color_mode = COLOR_MODE, 
                                             validation_split = Valid_ratio,
                                             subset = 'validation',
                                             image_size=IMG_SIZE,
                                             batch_size=BATCH_SIZE,
                                             label_mode ='categorical',
                                             )

Found 80 files belonging to 4 classes.
Using 40 files for training.
Found 80 files belonging to 4 classes.
Using 40 files for validation.


Normalize data set

In [ ]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
#reshape_layer = tf.keras.layers.Reshape(3, 4)
train_normalized_ds_VGG16 = train_dataset_VGG16.map(lambda x, y: (normalization_layer(x), y))
validation_normalized_ds_VGG16 = Validation_dataset_VGG16.map(lambda x, y: (normalization_layer(x), y))

Convert normalized dataset to images and labels in nparray 

In [ ]:
train_images_VGG16,train_labels_VGG16  = list(train_normalized_ds_VGG16.as_numpy_iterator())[0]
validate_images_VGG16,validate_labels_VGG16  = list(validation_normalized_ds_VGG16.as_numpy_iterator())[0]

In [ ]:
#tf.keras.layers.experimental.preprocessing.Resizing( 293, 293, interpolation="bilinear", crop_to_aspect_ratio=False)
#train_images_GoogleNet = train_images.resize(293,)

Generate model

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
model_VGG16 = VGG16(weights='imagenet') #,include_top=False
#model.summary()

Create Feautre Arrays

In [ ]:
TrainFeaturesArr_VGG16 = model_VGG16.predict(train_images_VGG16)
ValidateFeaturesArr_VGG16 = model_VGG16.predict(validate_images_VGG16)

Add KNN layer with K = 3 and P = 1 

In [ ]:
from sklearn import neighbors
knn_VGG16 = neighbors.KNeighborsClassifier(n_neighbors = 3, p = 1)
knn_VGG16.fit(TrainFeaturesArr_VGG16, train_labels_VGG16)
y_pred_VGG16 = knn_VGG16.predict(ValidateFeaturesArr_VGG16)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
print('Accuracy score: ',accuracy_score(y_pred_VGG16, validate_labels_VGG16))
print('F1 Score: ',f1_score(y_pred_VGG16, validate_labels_VGG16, average="macro"))

Accuracy score:  0.825
F1 Score:  0.845711500974659


# 2- Pretrained GoogleNet (InceptionV3)
using InceptionV3 pretrained with imagenet

Prepare Train and Validation data:

set ratio of 0.5 validation = 20 * 0.5 = 10 validation

Set Image size to 299*299 * 3 to support InceptionV3


In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
BATCH_SIZE = 128
IMG_SIZE = (299, 299)
COLOR_MODE = 'rgb' #"grayscale"  #  "rgb"
Color_Num = 3
Valid_ratio = 0.5

train_dataset_InceptionV3 = image_dataset_from_directory(PATH,
                                             seed = 1234,                                           
                                             color_mode = COLOR_MODE, 
                                             validation_split = Valid_ratio,
                                             subset = "training",
                                             image_size=IMG_SIZE,
                                             batch_size=BATCH_SIZE,
                                             label_mode ='categorical',
                                             )
Validation_dataset_InceptionV3 = image_dataset_from_directory(PATH,
                                             seed = 1234,                                           
                                             color_mode = COLOR_MODE, 
                                             validation_split = Valid_ratio,
                                             subset = 'validation',
                                             image_size=IMG_SIZE,
                                             batch_size=BATCH_SIZE,
                                             label_mode ='categorical',
                                             )

Found 80 files belonging to 4 classes.
Using 40 files for training.
Found 80 files belonging to 4 classes.
Using 40 files for validation.


Normalize data set

In [ ]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
#reshape_layer = tf.keras.layers.Reshape(3, 4)
train_normalized_ds_InceptionV3 = train_dataset_InceptionV3.map(lambda x, y: (normalization_layer(x), y))
validation_normalized_ds_InceptionV3 = Validation_dataset_InceptionV3.map(lambda x, y: (normalization_layer(x), y))

Convert normalized dataset to images and labels in nparray 

In [ ]:
train_images_InceptionV3,train_labels_InceptionV3  = list(train_normalized_ds_InceptionV3.as_numpy_iterator())[0]
validate_images_InceptionV3,validate_labels_InceptionV3  = list(validation_normalized_ds_InceptionV3.as_numpy_iterator())[0]

Generate model

In [ ]:
from tensorflow.keras.applications.inception_v3  import InceptionV3
model_InceptionV3 = InceptionV3(weights='imagenet') #,include_top=False
#model.summary()

Create Feautre Arrays

In [ ]:
TrainFeaturesArr_InceptionV3 = model_InceptionV3.predict(train_images_InceptionV3)
ValidateFeaturesArr_InceptionV3 = model_InceptionV3.predict(validate_images_InceptionV3)

Add KNN layer with K = 3 and P = 1 

In [ ]:
from sklearn import neighbors
knn_InceptionV3 = neighbors.KNeighborsClassifier(n_neighbors = 3, p = 1)
knn_InceptionV3.fit(TrainFeaturesArr_InceptionV3, train_labels_InceptionV3)
y_pred_InceptionV3 = knn_InceptionV3.predict(ValidateFeaturesArr_InceptionV3)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
print('InceptionV3 Accuracy score: ',accuracy_score(y_pred_InceptionV3, validate_labels_InceptionV3))
print('InceptionV3 F1 Score: ',f1_score(y_pred_InceptionV3, validate_labels_InceptionV3, average="macro"))

InceptionV3 Accuracy score:  0.975
InceptionV3 F1 Score:  0.9891304347826086


In [ ]:
#featuresArr = []
#for x in train_images:
#  features = model.predict(x)
#  features_reduce =  features.squeeze()
#  featuresArr.append(features_reduce)

In [ ]:
#from tensorflow.keras.applications.vgg16 import decode_predictions

#urllib.request.urlretrieve("https://user-images.githubusercontent.com/26264000/81228345-f167f600-8fbb-11ea-8722-d25dcda78a0c.jpg", "sample.jpg")
#img_path = 'sample.jpg'
#img = image.load_img(img_path, target_size=(224, 224))
#x = image.img_to_array(img)
#x = np.expand_dims(x, axis=0)
#x = preprocess_input(x)

#features = model.predict(x)
#print('Predicted:', decode_predictions(features, top=3)[0])